# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
        
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    
    nhl_df_2018 = nhl_df[nhl_df['year'] == 2018]
    nhl_df_2018 = nhl_df_2018.drop([0, 9 ,18 ,26])
    nhl_df_2018['W'] = nhl_df_2018['W'].astype(float)
    nhl_df_2018['L'] = nhl_df_2018['L'].astype(float)
    nhl_df_2018.reset_index(drop=True, inplace=True)
    
    win_loss = pd.DataFrame(columns=['NHL', 'win/loss'])

    for i in range(len(nhl_df_2018)):
        area = ' '.join(nhl_df_2018['team'][i].split()[0:-1])
        team = nhl_df_2018['team'][i].split()[-1].replace('*', '')
        ratio = nhl_df_2018['W'][i] / (nhl_df_2018['W'][i] + nhl_df_2018['L'][i])
        win_loss = win_loss.append({'NHL': team, 'win/loss': ratio}, ignore_index = True)
        
    win_loss = win_loss.replace({'NHL': {'Leafs': 'Maple Leafs', 'Wings': 'Red Wings', 'Jackets': 'Blue Jackets', 'Knights': 'Golden Knights'}})
    cities1 = cities.loc[:, ['Metropolitan area', 'Population (2016 est.)[8]', 'NHL']]
    cities1.rename(columns={'Metropolitan area': 'Area', 'Population (2016 est.)[8]': 'Population'}, inplace=True)
    
    def remove(data):
        j = ''.join([i for i in data if not i.isdigit()])
        i = j.find('[')
        if i > -1: j = j[:i]
        return j.strip()
    
    cities1['NHL'] = cities1['NHL'].apply(remove)
    
    cities1 = cities1.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'NHL': re.findall('[A-Z][^A-Z]*', cities1['NHL'][0])[0]}, 
                             ignore_index=True)
    cities1 = cities1.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'NHL': re.findall('[A-Z][^A-Z]*', cities1['NHL'][0])[1]}, 
                             ignore_index=True)
    cities1 = cities1.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'NHL': re.findall('[A-Z][^A-Z]*', cities1['NHL'][0])[2]}, 
                             ignore_index=True)
    cities1 = cities1.append({'Area': 'Los Angeles', 
                              'Population': 13310447, 
                              'NHL': re.findall('[A-Z][^A-Z]*', cities1['NHL'][1])[0]}, 
                             ignore_index=True)
    cities1 = cities1.append({'Area': 'Los Angeles', 
                              'Population': 13310447, 
                              'NHL': re.findall('[A-Z][^A-Z]*', cities1['NHL'][1])[1]}, 
                             ignore_index=True)
    cities1['Population'] = cities1['Population'].astype(float)
    
    df = pd.merge(win_loss, cities1, on='NHL')
    win_loss_by_region = df.groupby('Area')['win/loss'].mean()
    population_by_region = df.groupby('Area')['Population'].mean()
    

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
    nba_df_2018 = nba_df[nba_df['year'] == 2018]
    nba_df_2018['W'] = nba_df_2018['W'].astype(float)
    nba_df_2018['L'] = nba_df_2018['L'].astype(float)
    nba_df_2018 = nba_df_2018[['team', 'W', 'L']]
    
    def remove_r(data):
        j = ''.join([i for i in data])
        i = j.find('(')
        if i > -1: j = j[:i]
        return j.replace('*', '')
    
    nba_df_2018['team'] = nba_df_2018['team'].apply(remove_r)
    nba_win_loss = pd.DataFrame()
    
    for i in range(len(nba_df_2018)):
        team = nba_df_2018['team'][i].split()[-1]
        ratio = nba_df_2018['W'][i] / (nba_df_2018['W'][i] + nba_df_2018['L'][i])
        nba_win_loss = nba_win_loss.append({'NBA': team, 'win/loss': ratio}, ignore_index=True)
        
    nba_win_loss['NBA'][17] = 'Trail Blazers'
    
    cities2 = cities.loc[:, ['Metropolitan area', 'Population (2016 est.)[8]', 'NBA']]
    cities2.rename(columns={'Metropolitan area': 'Area', 'Population (2016 est.)[8]': 'Population'}, inplace=True)
    
    def remove_s(data):
        j = ''.join([i for i in data])
        i = j.find('[')
        if i > -1: j = j[:i]
        return j.strip()
    
    cities2['NBA'] = cities2['NBA'].apply(remove_s)
    
    cities2 = cities2.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'NBA': re.findall('[A-Z][^A-Z]*', cities2['NBA'][0])[0]}, 
                             ignore_index=True)
    cities2 = cities2.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'NBA': re.findall('[A-Z][^A-Z]*', cities2['NBA'][0])[1]}, 
                             ignore_index=True)

    cities2 = cities2.append({'Area': 'Los Angeles', 
                              'Population': 13310447, 
                              'NBA': re.findall('[A-Z][^A-Z]*', cities2['NBA'][1])[0]}, 
                             ignore_index=True)
    cities2 = cities2.append({'Area': 'Los Angeles', 
                              'Population': 13310447, 
                              'NBA': re.findall('[A-Z][^A-Z]*', cities2['NBA'][1])[1]}, 
                             ignore_index=True)
    
    cities2['Population'] = cities2['Population'].astype(float)
    
    df2 = pd.merge(nba_win_loss, cities2, on='NBA')
    win_loss_by_region = df2.groupby('Area')['win/loss'].mean()
    population_by_region = df2.groupby('Area')['Population'].mean()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [3]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]
    
    mlb_df_2018 = mlb_df[mlb_df['year'] == 2018]
    mlb_df_2018 = mlb_df_2018[['team', 'W', 'L']]
    mlb_df_2018['W'] = mlb_df_2018['W'].astype(float)
    mlb_df_2018['L'] = mlb_df_2018['L'].astype(float)
    
    mlb_win_loss = pd.DataFrame()
    for i in range(len(mlb_df_2018)):
        team = mlb_df_2018['team'][i].split()[-1].replace('*', '')
        ratio = mlb_df_2018['W'][i] / (mlb_df_2018['W'][i] + mlb_df_2018['L'][i])
        mlb_win_loss = mlb_win_loss.append({'MLB': team, 'win/loss': ratio}, ignore_index=True)
    
    mlb_win_loss['MLB'][0] = 'Red Sox'
    mlb_win_loss['MLB'][3] = 'Blue Jays'
    mlb_win_loss['MLB'][8] = 'White Sox'
    
    cities3 = cities.loc[:, ['Metropolitan area', 'Population (2016 est.)[8]', 'MLB']]
    cities3.rename(columns={'Metropolitan area': 'Area', 'Population (2016 est.)[8]': 'Population'}, inplace=True)
    
    def remove_s(data):
        j = ''.join([i for i in data])
        i = j.find('[')
        if i > -1: j = j[:i]
        return j.strip()
    
    cities3['MLB'] = cities['MLB'].apply(remove_s)
    
    cities3 = cities3.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'MLB': re.findall('[A-Z][^A-Z]*', cities3['MLB'][0])[0]}, 
                             ignore_index=True)
    cities3 = cities3.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'MLB': re.findall('[A-Z][^A-Z]*', cities3['MLB'][0])[1]}, 
                             ignore_index=True)

    cities3 = cities3.append({'Area': 'Los Angeles', 
                              'Population': 13310447, 
                              'MLB': re.findall('[A-Z][^A-Z]*', cities3['MLB'][1])[0]}, 
                             ignore_index=True)
    cities3 = cities3.append({'Area': 'Los Angeles', 
                              'Population': 13310447, 
                              'MLB': re.findall('[A-Z][^A-Z]*', cities3['MLB'][1])[1]}, 
                             ignore_index=True)

    cities3 = cities3.append({'Area': 'San Francisco Bay Area', 
                              'Population': 6657982, 
                              'MLB': re.findall('[A-Z][^A-Z]*', cities3['MLB'][2])[0]}, 
                             ignore_index=True)
    cities3 = cities3.append({'Area': 'San Francisco Bay Area', 
                              'Population': 6657982, 
                              'MLB': re.findall('[A-Z][^A-Z]*', cities3['MLB'][2])[1]}, 
                             ignore_index=True)

    cities3 = cities3.append({'Area': 'Chicago', 
                              'Population': 9512999, 
                              'MLB': re.findall('[A-Z][^A-Z]*', cities3['MLB'][3])[0]}, 
                             ignore_index=True)
    cities3 = cities3.append({'Area': 'Chicago', 
                              'Population': 9512999, 
                              'MLB': 'White Sox'}, 
                             ignore_index=True)
    cities3['Population'] = cities3['Population'].astype(float)
    
    df3 = pd.merge(mlb_win_loss, cities3, on='MLB')
    win_loss_by_region = df3.groupby('Area')['win/loss'].mean()
    population_by_region = df3.groupby('Area')['Population'].mean()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [4]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    
    population_by_region = [] # pass in metropolitan area population from cities
    win_loss_by_region = [] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]
    
    nfl_df_2018 = nfl_df[nfl_df['year'] == 2018]
    nfl_df_2018 = nfl_df_2018[['team', 'W', 'L']]
    nfl_df_2018 = nfl_df_2018.drop([0, 5, 10, 15, 20, 25, 30, 35])
    nfl_df_2018['W'] = nfl_df_2018['W'].astype(float)
    nfl_df_2018['L'] = nfl_df_2018['L'].astype(float)
    nfl_df_2018.reset_index(drop=True, inplace=True)
    
    nfl_win_loss = pd.DataFrame()
    for i in range(len(nfl_df_2018)):
        area = ' '.join(nfl_df_2018['team'][i].split()[0: -1])
        team = nfl_df_2018['team'][i].split()[-1].replace('*', '').replace('+', '')
        ratio = nfl_df_2018['W'][i] / (nfl_df_2018['W'][i] / nfl_df_2018['L'][i])
        nfl_win_loss = nfl_win_loss.append({'NFL': team, 'win/loss': ratio}, ignore_index=True)
    
    cities4 = cities.loc[:, ['Metropolitan area', 'Population (2016 est.)[8]', 'NFL']]
    cities4.rename(columns={'Metropolitan area': 'Area', 'Population (2016 est.)[8]': 'Population'}, inplace=True)
    
    def remove_s(data):
        j = ''.join([i for i in data])
        i = j.find('[')
        if i > -1: j = j[:i]
        return j.strip()
    
    cities4['NFL'] = cities['NFL'].apply(remove_s)
    
    cities4 = cities4.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'NFL': re.findall('[A-Z][^A-Z]*', cities4['NFL'][0])[0]}, 
                             ignore_index=True)
    cities4 = cities4.append({'Area': 'New York City', 
                              'Population': 20153634, 
                              'NFL': re.findall('[A-Z][^A-Z]*', cities4['NFL'][0])[1]}, 
                             ignore_index=True)

    cities4 = cities4.append({'Area': 'Los Angeles', 
                              'Population': 13310447, 
                              'NFL': re.findall('[A-Z][^A-Z]*', cities4['NFL'][1])[0]}, 
                             ignore_index=True)
    cities4 = cities4.append({'Area': 'Los Angeles', 
                              'Population': 13310447, 
                              'NFL': re.findall('[A-Z][^A-Z]*', cities4['NFL'][1])[1]}, 
                             ignore_index=True)

    cities4 = cities4.append({'Area': 'San Francisco Bay Area', 
                              'Population': 6657982, 
                              'NFL': re.findall('[A-Z][^A-Z]*', cities4['NFL'][2])[0]}, 
                             ignore_index=True)
    cities4 = cities4.append({'Area': 'San Francisco Bay Area', 
                              'Population': 6657982, 
                              'NFL': '49ers'}, 
                             ignore_index=True)
    cities4['Population'] = cities4['Population'].astype(float)
    
    df4 = pd.merge(nfl_win_loss, cities4, on='NFL')
    win_loss_by_region = df4.groupby('Area')['win/loss'].mean()
    population_by_region = df4.groupby('Area')['Population'].mean()
    
    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [5]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    # YOUR CODE HERE
    raise NotImplementedError()
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values